# 🎨 Stable Diffusion XL Image Generator

Generate high-quality images with Stable Diffusion XL (SDXL) using Colab's free GPU. No installation or API key required.

## Model Information
This notebook uses SDXL 1.0, offering:
- Higher resolution outputs (1024×1024)
- Better composition and coherence
- Improved prompt understanding
- Advanced features like LoRA support

## Features
- State-of-the-art image generation
- Simple interface with prompt input
- Multiple generation settings
- Negative prompts support
- Optional SDXL refiner for enhanced details
- Support for custom LoRA models

## Setup
First, let's install the required packages:

In [ ]:
!pip install -q diffusers==0.27.2 \
    transformers==4.39.3 \
    accelerate==0.28.0 \
    gradio==4.29.0 \
    torch \
    xformers \
    bitsandbytes \
    huggingface_hub==0.20.3

## Import Dependencies

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from transformers import BitsAndBytesConfig
import gradio as gr
from PIL import Image

## Load Models

In [ ]:
# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

# Base model
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    use_safetensors=True,
    device_map="auto",
    quantization_config=bnb_config
)

# Refiner model (load only when needed)
refiner = None

def load_refiner():
    global refiner
    if refiner is None:
        refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-refiner-1.0",
            variant="fp16",
            use_safetensors=True,
            device_map="auto",
            quantization_config=bnb_config
        )

# Enable memory efficient attention
pipe.enable_xformers_memory_efficient_attention()

## Create Generation Function

In [ ]:
def generate_image(prompt, negative_prompt="", num_steps=30, guidance_scale=7.5, 
                  use_refiner=False, refiner_steps=10, lora_model="", lora_scale=0.5,
                  advanced_mode=False):
    try:
        # Initialize LoRA tracking if not exists
        if not hasattr(generate_image, "current_lora"):
            generate_image.current_lora = ""
        
        # Load LoRA if specified and different from current
        cross_attention_kwargs = {}
        if advanced_mode and lora_model and lora_model != generate_image.current_lora:
            try:
                pipe.unet.load_attn_procs(lora_model)
                cross_attention_kwargs = {"scale": lora_scale}
                generate_image.current_lora = lora_model
            except Exception as e:
                print(f"Error loading LoRA model: {e}")
                return Image.new("RGB", (512, 512), color="black")
        
        # Generate base image
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale,
            cross_attention_kwargs=cross_attention_kwargs
        ).images[0]
        
        # Apply refiner if enabled
        if advanced_mode and use_refiner:
            load_refiner()
            image = refiner(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=refiner_steps,
                image=image
            ).images[0]
        
        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return Image.new("RGB", (512, 512), color="black")

## Create Gradio Interface

In [ ]:
with gr.Blocks() as interface:
    gr.Markdown(
        """
        # Stable Diffusion XL Image Generator
        Generate high-quality images from text descriptions using SDXL.
        """
    )
    
    with gr.Row():
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="Prompt",
                placeholder="Enter your image description here...",
                lines=3
            )
            negative_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="What you don't want in the image..."
            )
            
            with gr.Row():
                num_steps = gr.Slider(
                    minimum=1,
                    maximum=50,
                    value=30,
                    step=1,
                    label="Number of Steps"
                )
                guidance_scale = gr.Slider(
                    minimum=1,
                    maximum=20,
                    value=7.5,
                    step=0.5,
                    label="Guidance Scale"
                )
            
            advanced_mode = gr.Checkbox(label="Advanced Mode", value=False)
            
            advanced_options = gr.Column(visible=False)
            with advanced_options:
                use_refiner = gr.Checkbox(label="Use Refiner", value=False)
                refiner_steps = gr.Slider(
                    minimum=1,
                    maximum=50,
                    value=10,
                    step=1,
                    label="Refiner Steps"
                )
                lora_model = gr.Textbox(
                    label="LoRA Model ID",
                    placeholder="e.g., ostris/super-realistic-xl"
                )
                lora_scale = gr.Slider(
                    minimum=0,
                    maximum=1,
                    value=0.5,
                    step=0.1,
                    label="LoRA Scale"
                )
            
            advanced_mode.change(
                lambda x: gr.update(visible=x),
                inputs=advanced_mode,
                outputs=advanced_options
            )
        
        with gr.Column(scale=2):
            output = gr.Image(label="Generated Image")
    
    gr.Examples([
        ["A majestic lion in the savanna at sunset, highly detailed, professional photography", 
         "blurry, low quality", 30, 7.5, False, 10, "", 0.5, False],
        ["A futuristic cityscape with flying cars and neon lights, cyberpunk style", 
         "grainy, ugly, distorted", 30, 7.5, True, 10, "", 0.5, True]
    ], [prompt, negative_prompt, num_steps, guidance_scale, use_refiner, 
        refiner_steps, lora_model, lora_scale, advanced_mode])
    
    inputs = [prompt, negative_prompt, num_steps, guidance_scale, use_refiner, 
              refiner_steps, lora_model, lora_scale, advanced_mode]
    
    btn = gr.Button("Generate")
    btn.click(fn=generate_image, inputs=inputs, outputs=output)

interface.launch(share=True)